# Supervised Learning

Given the **train.csv** and **test.csv**, perform the following actions.
1. Create classifiers from train.csv data, in order to predict ``att10`` as label based on the
rest of the attributes. Use at least 5 algorithms (k-NN, Decision Tree, Logistic
Regression, Voting, Averaging, Bagging, Random Forest, Averaging, Voting, AdaBoost,
XGBoost, LightGBM, CatBoost, or Stacking). Use AUC for your model evaluation
performance.
Submit your .ipynb file that contains your coding process in creating classifiers.
2. Choose the best Classifier based on highest AUC and use it for predicting the test.csv
data.
Submit your .csv file that contains the prediction of test.csv data. Sample of prediction
can be found in **sample_prediction.csv** file. 

In [2]:
# Import the libraries and load the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
pd.options.display.max_columns = None
pd.options.display.max_rows = 15

data_train = pd.read_csv(r'dataset/train.csv')
data_test = pd.read_csv(r'dataset/test.csv')

display(data_train.sample(5))
display(data_test.sample(5))

print('Train data shape {}'.format(data_train.shape))
print('Test data shape {}'.format(data_test.shape))

,att1,att2,att3,att4,att5,att6,att7,att8a,att8b,att8c,att8d,att8e,att8f,att8g,att8h,att8i,att8j,att9a,att9b,att9c,att10
1565,0.91,0.55,3,223,3,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
7395,0.84,0.89,4,187,2,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
6636,0.50,0.40,3,180,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
499,0.59,0.72,3,182,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2627,0.69,0.82,4,137,2,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


,att1,att2,att3,att4,att5,att6,att7,att8a,att8b,att8c,att8d,att8e,att8f,att8g,att8h,att8i,att8j,att9a,att9b,att9c
855,0.18,0.46,4,249,4,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
860,0.59,0.73,3,172,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
259,0.85,0.86,3,255,2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1632,0.10,0.84,6,261,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1369,0.87,0.98,4,173,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


Train data shape (8000, 21)
Test data shape (2000, 20)


We can see that the training and test data already have the same columns, except that the label column `att10` is missing on the test data.

In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
att1     8000 non-null float64
att2     8000 non-null float64
att3     8000 non-null int64
att4     8000 non-null int64
att5     8000 non-null int64
att6     8000 non-null int64
att7     8000 non-null int64
att8a    8000 non-null int64
att8b    8000 non-null int64
att8c    8000 non-null int64
att8d    8000 non-null int64
att8e    8000 non-null int64
att8f    8000 non-null int64
att8g    8000 non-null int64
att8h    8000 non-null int64
att8i    8000 non-null int64
att8j    8000 non-null int64
att9a    8000 non-null int64
att9b    8000 non-null int64
att9c    8000 non-null int64
att10    8000 non-null int64
dtypes: float64(2), int64(19)
memory usage: 1.3 MB


All of the features are numerical and have no missing values, and as we can infer that the `att8` and `att9` are the results of one hot encode of a categorical feature.

In [21]:
display(data_train[data_train.columns[:7]].describe())

,att1,att2,att3,att4,att5,att6,att7
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,0.612260,0.717594,3.791375,201.245000,3.486375,0.145375,0.020625
std,0.248338,0.170957,1.230463,49.837947,1.446055,0.352501,0.142134
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,201.000000,3.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000


In [43]:
display(data_train['att10'].value_counts(normalize=True))

0    0.760125
1    0.239875
Name: att10, dtype: float64

In [72]:
from sklearn.metrics import fbeta_score, roc_auc_score
from time import time

def train_predict(learner, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # Fit the learner to the training data 
    start = time() # Get start time
    learner.fit(X_train, y_train)
    end = time() # Get end time
    
    # Save the model
    results['model'] = learner
    
    # Calculate the training time
    results['train_time'] = end - start
        
    # Get the predictions on the test set(X_test),
    # then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    # Calculate the total prediction time
    results['pred_time'] = end - start
            
    # Compute roc_auc on the first 300 training samples which is y_train[:300]
    results['roc_auc_train'] = roc_auc_score(y_train[:300], predictions_train)
        
    # Compute roc_auc on test set using accuracy_score()
    results['roc_auc_test'] = roc_auc_score(y_test, predictions_test)
    
    # Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, beta=0.5)
        
    # Compute F-score on the test set which is y_test
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, len(X_train)))
        
    # Return the results
    return results

In [69]:
from sklearn.model_selection import KFold

# Drop the last column of each one hot encoded features
X = data_train.drop(['att8j', 'att9c', 'att10'], axis=1)
X = np.array(X)
y = data_train['att10']

kf = KFold(n_splits=5, shuffle=True)


In [73]:
# Import the five supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB 
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import catboost as cb

# Initialize the five models
clf_A = LogisticRegression()
clf_B = MultinomialNB()
clf_C = DecisionTreeClassifier()
clf_D = xgb.XGBClassifier()
clf_E = cb.CatBoostClassifier()

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    count = 1
    for train, valid in kf.split(X):
        results[clf_name][count] = train_predict(clf, X[train], y[train], X[valid], y[valid])
        count+=1

C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mpadmanagara\AppData\Local\Continuum\anacon

LogisticRegression trained on 6400 samples.
LogisticRegression trained on 6400 samples.
LogisticRegression trained on 6400 samples.
LogisticRegression trained on 6400 samples.
LogisticRegression trained on 6400 samples.
MultinomialNB trained on 6400 samples.
MultinomialNB trained on 6400 samples.
MultinomialNB trained on 6400 samples.
MultinomialNB trained on 6400 samples.
MultinomialNB trained on 6400 samples.
DecisionTreeClassifier trained on 6400 samples.

C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



DecisionTreeClassifier trained on 6400 samples.
DecisionTreeClassifier trained on 6400 samples.
DecisionTreeClassifier trained on 6400 samples.
DecisionTreeClassifier trained on 6400 samples.
XGBClassifier trained on 6400 samples.
XGBClassifier trained on 6400 samples.
XGBClassifier trained on 6400 samples.
XGBClassifier trained on 6400 samples.
XGBClassifier trained on 6400 samples.
Learning rate set to 0.028332
0:	learn: 0.6458392	total: 6.29ms	remaining: 6.29s
1:	learn: 0.6028726	total: 12.5ms	remaining: 6.22s
2:	learn: 0.5660619	total: 17.9ms	remaining: 5.96s
3:	learn: 0.5282945	total: 23.6ms	remaining: 5.87s
4:	learn: 0.4930996	total: 34.1ms	remaining: 6.78s
5:	learn: 0.4620377	total: 41ms	remaining: 6.8s
6:	learn: 0.4348961	total: 48.1ms	remaining: 6.82s
7:	learn: 0.4110067	total: 55ms	remaining: 6.81s
8:	learn: 0.3869799	total: 60.8ms	remaining: 6.7s
9:	learn: 0.3660858	total: 69.7ms	remaining: 6.9s
10:	learn: 0.3470746	total: 76.4ms	remaining: 6.87s
11:	learn: 0.3288842	total:

167:	learn: 0.0751175	total: 1.2s	remaining: 5.93s
168:	learn: 0.0750101	total: 1.2s	remaining: 5.92s
169:	learn: 0.0748796	total: 1.21s	remaining: 5.92s
170:	learn: 0.0747074	total: 1.22s	remaining: 5.91s
171:	learn: 0.0745666	total: 1.22s	remaining: 5.9s
172:	learn: 0.0744172	total: 1.23s	remaining: 5.89s
173:	learn: 0.0742171	total: 1.24s	remaining: 5.88s
174:	learn: 0.0740968	total: 1.25s	remaining: 5.87s
175:	learn: 0.0739975	total: 1.25s	remaining: 5.87s
176:	learn: 0.0738332	total: 1.26s	remaining: 5.86s
177:	learn: 0.0736920	total: 1.27s	remaining: 5.86s
178:	learn: 0.0734201	total: 1.27s	remaining: 5.85s
179:	learn: 0.0732172	total: 1.28s	remaining: 5.85s
180:	learn: 0.0730977	total: 1.29s	remaining: 5.84s
181:	learn: 0.0729411	total: 1.3s	remaining: 5.84s
182:	learn: 0.0727873	total: 1.31s	remaining: 5.83s
183:	learn: 0.0724607	total: 1.31s	remaining: 5.83s
184:	learn: 0.0722607	total: 1.32s	remaining: 5.83s
185:	learn: 0.0721432	total: 1.33s	remaining: 5.82s
186:	learn: 0.07

...

333:	learn: 0.0554967	total: 2.41s	remaining: 4.8s
334:	learn: 0.0554200	total: 2.42s	remaining: 4.8s
335:	learn: 0.0552506	total: 2.42s	remaining: 4.79s
336:	learn: 0.0551518	total: 2.43s	remaining: 4.79s
337:	learn: 0.0550551	total: 2.44s	remaining: 4.78s
338:	learn: 0.0549406	total: 2.45s	remaining: 4.77s
339:	learn: 0.0548955	total: 2.45s	remaining: 4.76s
340:	learn: 0.0548044	total: 2.46s	remaining: 4.75s
341:	learn: 0.0547124	total: 2.47s	remaining: 4.75s
342:	learn: 0.0546249	total: 2.47s	remaining: 4.74s
343:	learn: 0.0544992	total: 2.48s	remaining: 4.73s
344:	learn: 0.0544376	total: 2.49s	remaining: 4.72s
345:	learn: 0.0543350	total: 2.49s	remaining: 4.71s
346:	learn: 0.0542489	total: 2.5s	remaining: 4.7s
347:	learn: 0.0541045	total: 2.51s	remaining: 4.7s
348:	learn: 0.0540145	total: 2.51s	remaining: 4.69s
349:	learn: 0.0539001	total: 2.52s	remaining: 4.68s
350:	learn: 0.0538250	total: 2.53s	remaining: 4.67s
351:	learn: 0.0536634	total: 2.53s	remaining: 4.66s
352:	learn: 0.053

500:	learn: 0.0420994	total: 3.64s	remaining: 3.63s
501:	learn: 0.0420395	total: 3.65s	remaining: 3.62s
502:	learn: 0.0419818	total: 3.65s	remaining: 3.61s
503:	learn: 0.0419373	total: 3.66s	remaining: 3.6s
504:	learn: 0.0418245	total: 3.67s	remaining: 3.6s
505:	learn: 0.0417497	total: 3.68s	remaining: 3.59s
506:	learn: 0.0416948	total: 3.68s	remaining: 3.58s
507:	learn: 0.0416194	total: 3.69s	remaining: 3.57s
508:	learn: 0.0415691	total: 3.7s	remaining: 3.57s
509:	learn: 0.0415036	total: 3.71s	remaining: 3.57s
510:	learn: 0.0414306	total: 3.72s	remaining: 3.56s
511:	learn: 0.0413928	total: 3.73s	remaining: 3.55s
512:	learn: 0.0413240	total: 3.74s	remaining: 3.55s
513:	learn: 0.0412640	total: 3.74s	remaining: 3.54s
514:	learn: 0.0411938	total: 3.75s	remaining: 3.53s
515:	learn: 0.0411372	total: 3.76s	remaining: 3.53s
516:	learn: 0.0410831	total: 3.77s	remaining: 3.52s
517:	learn: 0.0410263	total: 3.77s	remaining: 3.51s
518:	learn: 0.0409727	total: 3.78s	remaining: 3.5s
519:	learn: 0.04

668:	learn: 0.0335578	total: 5.05s	remaining: 2.5s
669:	learn: 0.0335349	total: 5.05s	remaining: 2.49s
670:	learn: 0.0334881	total: 5.06s	remaining: 2.48s
671:	learn: 0.0334332	total: 5.07s	remaining: 2.48s
672:	learn: 0.0334031	total: 5.08s	remaining: 2.47s
673:	learn: 0.0333680	total: 5.09s	remaining: 2.46s
674:	learn: 0.0333033	total: 5.1s	remaining: 2.46s
675:	learn: 0.0332911	total: 5.12s	remaining: 2.45s
676:	learn: 0.0332553	total: 5.13s	remaining: 2.45s
677:	learn: 0.0332269	total: 5.13s	remaining: 2.44s
678:	learn: 0.0332027	total: 5.14s	remaining: 2.43s
679:	learn: 0.0330971	total: 5.15s	remaining: 2.42s
680:	learn: 0.0330627	total: 5.16s	remaining: 2.42s
681:	learn: 0.0330282	total: 5.16s	remaining: 2.41s
682:	learn: 0.0329581	total: 5.17s	remaining: 2.4s
683:	learn: 0.0329362	total: 5.18s	remaining: 2.39s
684:	learn: 0.0329194	total: 5.18s	remaining: 2.38s
685:	learn: 0.0328816	total: 5.19s	remaining: 2.38s
686:	learn: 0.0328024	total: 5.2s	remaining: 2.37s
687:	learn: 0.03

831:	learn: 0.0278090	total: 6.29s	remaining: 1.27s
832:	learn: 0.0277709	total: 6.29s	remaining: 1.26s
833:	learn: 0.0277205	total: 6.3s	remaining: 1.25s
834:	learn: 0.0277047	total: 6.31s	remaining: 1.25s
835:	learn: 0.0276773	total: 6.32s	remaining: 1.24s
836:	learn: 0.0276490	total: 6.33s	remaining: 1.23s
837:	learn: 0.0276027	total: 6.33s	remaining: 1.22s
838:	learn: 0.0275563	total: 6.34s	remaining: 1.22s
839:	learn: 0.0275448	total: 6.35s	remaining: 1.21s
840:	learn: 0.0275156	total: 6.36s	remaining: 1.2s
841:	learn: 0.0274885	total: 6.36s	remaining: 1.19s
842:	learn: 0.0274466	total: 6.37s	remaining: 1.19s
843:	learn: 0.0273820	total: 6.38s	remaining: 1.18s
844:	learn: 0.0273643	total: 6.38s	remaining: 1.17s
845:	learn: 0.0273488	total: 6.39s	remaining: 1.16s
846:	learn: 0.0273209	total: 6.4s	remaining: 1.16s
847:	learn: 0.0272748	total: 6.4s	remaining: 1.15s
848:	learn: 0.0272574	total: 6.41s	remaining: 1.14s
849:	learn: 0.0272280	total: 6.42s	remaining: 1.13s
850:	learn: 0.02

CatBoostClassifier trained on 6400 samples.


In [115]:
results

{'LogisticRegression': {1: {'model': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, l1_ratio=None, max_iter=100,
                      multi_class='warn', n_jobs=None, penalty='l2',
                      random_state=None, solver='warn', tol=0.0001, verbose=0,
                      warm_start=False),
   'train_time': 0.03500008583068848,
   'pred_time': 0.0009999275207519531,
   'roc_auc_train': 0.6357330782109543,
   'roc_auc_test': 0.6430668191917449,
   'f_train': 0.5341880341880342,
   'f_test': 0.5270972531551597},
  2: {'model': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, l1_ratio=None, max_iter=100,
                      multi_class='warn', n_jobs=None, penalty='l2',
                      random_state=None, solver='warn', tol=0.0001, verbose=0,
                      warm_start=False),
   'train_time': 0.01900005340576172,
   'pred_t

In [123]:
clf_min_time = {}
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E]:
    clf_name = clf.__class__.__name__
    clf_min_time[clf_name] = min([results[clf_name][i]["train_time"] for i in range(1, 6)])

In [109]:
clf_max_score = {}
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E]:
    clf_name = clf.__class__.__name__
    clf_max_score[clf_name] = max([results[clf_name][i]["roc_auc_test"] for i in range(1, 6)])

In [124]:
clf_min_time

{'LogisticRegression': 0.01699995994567871,
 'MultinomialNB': 0.0019998550415039062,
 'DecisionTreeClassifier': 0.020999908447265625,
 'XGBClassifier': 0.6319999694824219,
 'CatBoostClassifier': 7.491999626159668}

In [116]:
clf_max_score

{'LogisticRegression': 0.6622892306656228,
 'MultinomialNB': 0.5,
 'DecisionTreeClassifier': 0.9697420423214483,
 'XGBClassifier': 0.9628392609977735,
 'CatBoostClassifier': 0.9720688830028632}

In [126]:
print('Classifier with best score is {}'.format(max(clf_max_score, key=clf_max_score.get)))
print('Classifier with fastest training time is {}'.format(min(clf_min_time, key=clf_min_time.get)))

Classifier with best score is CatBoostClassifier
Classifier with fastest training time is MultinomialNB


From above result I will choose the `XGBClassifier` because the score and the training time is good.

In [129]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [132]:
# Import 'GridSearchCV', 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

# Initialize the classifier
clf = xgb.XGBClassifier(random_state=42)

# Created the parameters dict for xgb classifier
parameters = {
    'max_depth':range(1, 10), 
    'learning_rate':[0.01, 0.05, 0.1],      
    'gamma':range(0, 5)
}

# Make an roc_auc scoring object using make_scorer()
scorer = make_scorer(roc_auc_score)

# Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

C:\Users\mpadmanagara\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Unoptimized model
------


NameError: name 'accuracy_score' is not defined

In [133]:
# Report the before-and-afterscores
print("Unoptimized model\n------")
print("ROC_AUC score on testing data: {:.4f}".format(roc_auc_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final ROC_AUC score on the testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

Unoptimized model
------
ROC_AUC score on testing data: 0.9647
F-score on testing data: 0.9649

Optimized Model
------
Final ROC_AUC score on the testing data: 0.9726
Final F-score on the testing data: 0.9801


In [143]:
features_columns = data_train.drop(['att8j', 'att9c', 'att10'], axis=1).columns
col_importance_score = {}
for i in range(len(features_columns)):
    col_importance_score[features_columns[i]] = best_clf.feature_importances_[i]

In [146]:
col_importance_score

{'att1': 0.2954557,
 'att2': 0.07647457,
 'att3': 0.13411395,
 'att4': 0.045030687,
 'att5': 0.23844256,
 'att6': 0.023917245,
 'att7': 0.013406309,
 'att8a': 0.012392218,
 'att8b': 0.010416632,
 'att8c': 0.009066616,
 'att8d': 0.024436198,
 'att8e': 0.018360063,
 'att8f': 0.0154176075,
 'att8g': 0.012611132,
 'att8h': 0.020124257,
 'att8i': 0.014575411,
 'att9a': 0.014196939,
 'att9b': 0.021561962}

In [149]:
print('Top five important features based on the best XGBClassifier: {}'\
.format(sorted(col_importance_score, key=col_importance_score.get, reverse=True)[:5]))

Top five important features based on the best XGBClassifier: ['att1', 'att5', 'att3', 'att2', 'att4']


In [169]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_reduced = data_train[['att1', 'att5', 'att3', 'att2', 'att4']]
X_train_reduced, X_test_reduced, y_train, y_test = train_test_split(X_reduced, y, random_state=42)

# Train on the "best" model found from grid search earlier
clf = clone(best_clf).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)
best_predictions = best_clf.predict(X_test)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("ROC_AUC on testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("ROC_AUC on testing data: {:.4f}".format(roc_auc_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

Final Model trained on full data
------
ROC_AUC on testing data: 0.9726
F-score on testing data: 0.9801

Final Model trained on reduced data
------
ROC_AUC on testing data: 0.9745
F-score on testing data: 0.9793


In [177]:
X = data_test.drop(['att8j', 'att9c'], axis=1)
X = np.array(X)
y_pred = best_clf.predict_proba(X)

In [225]:
data_pred = {'no':[i for i in range(1, len(y_pred)+1)], 'prediction_for_class_1':[i[1] for i in y_pred]}

In [226]:
df_pred = pd.DataFrame(data=data_pred)

In [229]:
df_pred.to_csv('answer.csv', index=False)